In [77]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [79]:
df_1 = pd.read_csv('x_user_data.csv')

In [81]:
df_2 = pd.read_csv('Bot_user_data2.csv')

In [83]:
retrain_com_df = pd.concat([df_1, df_2], ignore_index=True)

In [85]:
label_col = 'fake'
features = retrain_com_df.drop(columns=[label_col])
label = retrain_com_df[label_col]

scaler = StandardScaler()
scaler.fit(features)
normalized_features = scaler.transform(features)
df_normalized = pd.DataFrame(normalized_features, columns=features.columns)

df_normalized[label_col] = label
df_normalized.to_csv('retrain_z_score.csv', index=False)

print("완료")

완료


In [87]:
df_3 = pd.read_csv('train.csv')

In [89]:
df_4 = pd.read_csv("test.csv")

In [91]:
df_com = pd.concat([df_3,df_4], ignore_index = True)

In [93]:
label_col = 'fake'
test_features = df_com.drop(columns=[label_col])
test_label = df_com[label_col]

test_normalized_features = scaler.transform(test_features)
test_df_normal = pd.DataFrame(test_normalized_features, columns = test_features.columns)


In [95]:
test_df_normal[label_col] = test_label

In [97]:
test_df_normal.to_csv('model_b_prime_test.csv')

In [99]:
df_x =  df_normalized

In [101]:
from scipy.stats import ttest_ind

# 봇과 휴먼 계정으로 나누기
bot_df = df_x[df_x['fake'] == 1]
human_df = df_x[df_x['fake'] == 0]

# t-test 수행
t_test_results = []
for col in df_x.columns[:-1]:  # 'fake' 제외
    t_stat, p_value = ttest_ind(bot_df[col], human_df[col], equal_var=False)
    t_test_results.append({
        "feature": col,
        "t_statistic": t_stat,
        "p_value": p_value,
        "reject_null": p_value < 0.05
    })

# 결과를 DataFrame으로 변환하고 정렬
t_test_df = pd.DataFrame(t_test_results).sort_values(by="p_value")

t_test_df


,feature,t_statistic,p_value,reject_null
5,description length,-21.322280,8.245719e-96,True
8,#posts,-14.056530,5.152958e-43,True
1,nums/length username,9.264103,3.164647e-20,True
2,fullname words,-6.736396,1.850194e-11,True
7,private,5.216508,1.916712e-07,True
6,external URL,-4.392906,1.148084e-05,True
10,#follows,-3.950434,8.042372e-05,True
3,nums/length fullname,3.162030,1.580134e-03,True
9,#followers,-2.997420,2.753288e-03,True
4,name==username,0.038204,9.695270e-01,False


In [103]:

t_test_df.to_csv("model_b_prime_data_vali.csv", index=False)
